# Using WKT Location Data in ArcGIS
[Well-Known Text (WKT)](https://libgeos.org/specifications/wkt/) is a format that stores geometry information as a string within a table. When stored in this way, each row in the table (such as a CSV file) will include an attribute that describes the geometry of a shape, stored as a string value. 

The notation starts by defining the type of geometric shape (e.g. POINT, POLYGON). It then includes longitude and latitude coordinates for each relevant point in the shape (e.g. if it's a polygon, every vertex along the polygon boundary will be listed). 

ArcGIS does not have a direct way to import WKT data. But there is a fromWKT function within the `arcpy` library. This script demonstrates how to use it

In [ ]:
import arcpy
import os
import csv

In [ ]:
# set up your workspace
aprx = arcpy.mp.ArcGISProject("CURRENT")
default_folder = aprx.homeFolder
default_gdb = aprx.defaultGeodatabase  
arcpy.env.overwriteOutput = True
print(default_folder)
print(default_gdb)

## Loading data

For this illustration, I'll use a CSV of [neighborhood boundaries](https://data.cityofnewyork.us/resource/9nt8-h7nd.csv) from NYC Open Data. I'm going to simply download the CSV, name it NTA_boundaries.csv, and add it to the default folder identified above. But if you wanted to, you could pull this from the NYC Open Data API using sodapy. 

In [ ]:
# define the parameters (you'll need to update all of them for your data)
input_file = os.path.join(default_folder, "NTA_boundaries.csv")
output_name = "NTA_boundaries"
geom_type = "POLYGON"  
geom_prj = 4326  ## Assuming we're using WG 1984 as the projection

In [ ]:
## create an empty layer
output_layer = arcpy.management.CreateFeatureclass(out_path = default_gdb,
                                                   out_name = output_name,
                                                   geometry_type = geom_type,
                                                   spatial_reference = geom_prj)

## add a primary key field (whatever it's called in your data)
arcpy.AddField_management(output_layer, "NTA2020", "TEXT")

print(output_layer)

## Reading the WKT records

The next step is to identify the WKT and primary key fields, then draw the former to geomtry and insert the latter into the unique identifier row. After this is finished, you can join the original CSV to the new layer by that primary key. This is based in part on code from a 2023 [ESRI blog post](https://community.esri.com/t5/arcgis-pro-questions/how-to-convert-wkt-polygon-data-in-csv-file-into/td-p/1304292). 

In [ ]:
# Reads WKT and primary key records, and insert to the empty layer
with open(input_file, "r", newline="") as csvfile:
    csvreader = csv.DictReader(csvfile)

    ## insert the polygon and primary key data
    with arcpy.da.InsertCursor(output_layer, ["SHAPE@", "NTA2020"]) as icurs:
        for row in csvreader:
            wkt_geom = row["the_geom"]  ## change this to your WKT column name
            primary_key = row["NTA2020"]  ## change this to your unique ID column name

            ## Validate the WKT before processing
            if not wkt_geom or "POLYGON" not in wkt_geom:
                print(f"SKIPPING invalid WKT: {wkt_geom[:50]}")
                continue

            try:
                geom_from_wkt = arcpy.FromWKT(wkt_geom, arcpy.SpatialReference(4326))

                ## Adds the polygon and unique identifier
                icurs.insertRow([geom_from_wkt, primary_key])

            except (TypeError, RuntimeError) as e:
                print(f"SKIPPING due to conversion failure: {wkt_geom[:100]}... | {e}")
                continue

print("Done.")